In [1]:
import sys
sys.path.append('/dccstor/hoo-misha-1/wilds/WOODS')
#sys.path.append('/dccs|tor/hoo-misha-1/wilds/WOODS/scripts/')
sys.path.append('/dccstor/hoo-misha-1/wilds/wilds')
import numpy as np
import torch
import os

from scripts.utils import *

In [2]:
"python examples/run_expt.py --root_dir data --log_dir logs/wassersteindeepCORAL --dataset iwildcam --algorithm wassersteindeepCORAL --seed 0 --lr 3e-05 --weight_decay 0 --coral_penalty_weight 100.0"

'python examples/run_expt.py --root_dir data --log_dir logs/wassersteindeepCORAL --dataset iwildcam --algorithm wassersteindeepCORAL --seed 0 --lr 3e-05 --weight_decay 0 --coral_penalty_weight 100.0'

In [3]:
# dataset = 'breeds'
# for model in os.listdir(f'/dccstor/hoo-misha-1/wilds/wilds/pretrained/{dataset}'):
#     if dataset == 'iwildcam':
#         model = model[9:-4]
#     if dataset == 'breeds':
#         model = model[7:-4]
#     elif dataset == 'camelyon17':
#         model = model[11:-4]
#     if model not in ['wassersteindeepCORAL']:
#         continue
#     save_path = f'/dccstor/hoo-misha-1/wilds/wilds/pretrained/iwildcam/iwildcam_{model}.pth'
#     d = torch.load(save_path, map_location=torch.device('cpu'))
#     alg = d['algorithm']
#     if 'model.classifier.weight' in alg:
#         weight = alg['model.classifier.weight']
#         bias = alg['model.classifier.bias']
#     elif 'classifier.weight' in alg:
#         weight = alg['classifier.weight']
#         bias = alg['classifier.bias']
#     elif 'model.fc.bias' in alg:
#         weight = alg['model.fc.weight']
#         bias = alg['model.fc.bias']
#     elif 'model.1.weight' in alg:
#         weight = alg['model.1.weight']
#         bias = alg['model.1.bias']
#     else:
#         raise Exception(f"Cannot find classifier for {model}")
#     np.save(f'/dccstor/hoo-misha-1/wilds/WOODS/classifiers/iwildcam/iwildcam_{model}_classifier_weight.npy', weight)
#     np.save(f'/dccstor/hoo-misha-1/wilds/WOODS/classifiers/iwildcam/iwildcam_{model}_classifier_bias.npy', bias)


In [4]:
def runner_command(runner_name, dataset, model, sampler, args, wandb = False):
    if dataset == 'camelyon17':
        queue = 'x86_1h'
    else:
        # if 'iterative' in sampler:
        #     queue = 'x86_24h'
        if 'kmeans' in runner_name:
            queue = 'x86_6h'
        
        else:
            queue = 'x86_6h'
    if not wandb:
        return f'jbsub -q {queue} -cores 8 -mem 100g -proj {dataset} -name {model}_{sampler} python /dccstor/hoo-misha-1/wilds/WOODS/scripts/{runner_name}.py --no_rng --dataset {dataset} --model {model} --sampler {sampler} {args}'
    return f'jbsub -q {queue} -cores 8 -mem 100g -proj {dataset} -name {model}_{sampler} python /dccstor/hoo-misha-1/wilds/WOODS/scripts/{runner_name}.py --no_rng --dataset {dataset} --model {model} {args} --use_wandb --reset'

In [8]:
dataset = 'iwildcam'
sampling_algorithms_dict = {'balanced':'balanced_sample_ind', 'full':'full_kmeans_sample_ind', 'class':'class_kmeans_sample_ind' , 'typiclust':'0'}
for model in os.listdir(f'/dccstor/hoo-misha-1/wilds/wilds/pretrained/{dataset}'):
    if dataset == 'iwildcam':
        model = model[9:-4]
    if dataset == 'breeds':
        model = model[7:-4]
    elif dataset == 'camelyon17':
        model = model[11:-4]
    elif dataset == 'cifar100':
        model = model[9:-4]
    try: 
        os.makedirs(f'/dccstor/hoo-misha-1/wilds/WOODS/results/{dataset}/{model}')
    except:
        pass
    # for sampler in sampling_algorithms_dict:
    #     os.system(runner_command(runner_name='runner', dataset=dataset, model=model, sampler = sampler, args = ' '))
    # continue
    for sampler in ['iterative']:
        for weighted in ['--weighted', '']:
            for density in ['--density', '']:
                for precluster in ['--precluster', '']:
                    for typicality in ['--typicality', '']:
                        args = ' '.join([weighted, density, precluster, typicality])

                        os.system(runner_command(runner_name='kmeans_runner', dataset=dataset, model=model, sampler = sampler, args = args))

/opt/share/exec/jbsub8 -q x86_6h -cores 8 -mem 100g -proj breeds -name ERM_iterative python /dccstor/hoo-misha-1/wilds/WOODS/scripts/kmeans_runner.py --no_rng --dataset breeds --model ERM --sampler iterative --weighted --density --precluster --typicality
# bsub -q x86_6h -g /hbyang/_/breeds -J ERM_iterative -M 102400 -hl -n 8 -R "rusage[mem=108544] span[ptile=8] affinity[core(1)]" -oo /u/hbyang/.lsf/cccCluster/%J.stdout -eo /u/hbyang/.lsf/cccCluster/%J.stderr python /dccstor/hoo-misha-1/wilds/WOODS/scripts/kmeans_runner.py --no_rng --dataset breeds --model ERM --sampler iterative --weighted --density --precluster --typicality
Job <2883269> is submitted to queue <x86_6h>.
/opt/share/exec/jbsub8 -q x86_6h -cores 8 -mem 100g -proj breeds -name ERM_iterative python /dccstor/hoo-misha-1/wilds/WOODS/scripts/kmeans_runner.py --no_rng --dataset breeds --model ERM --sampler iterative --weighted --density --precluster
# bsub -q x86_6h -g /hbyang/_/breeds -J ERM_iterative -M 102400 -hl -n 8 -R "r